In [8]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


### Downloaded from wikipedia page using panda
### Converted into dataframe
### dropped all rows which had borough as not assigned

In [9]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
print(df)
df1 = df[df.Borough != 'Not assigned']
print(df1)

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
283      M8Z         Etobicoke              Mimico NW
284      M8Z         Etobicoke     The Queensway West
285      M8Z         Etobicoke  Royal York South West
286      M8Z         Etobicoke         South of Bloor
287      M9Z      Not assigned           Not assigned

[288 rows x 3 columns]
    Postcode           Borough             Neighbourhood
2        M3A        North York                 Parkwoods
3        M4A        North York          Victoria Village
4        M5A  Downtown Toronto              Harbourfront
5        M5A  Downtown Toronto               Regent Park
6        M6A        North York          Law

In [10]:
df2 = df1.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df2.columns = ['Postal Code', 'Borough', 'Neighbourhood']

### identified Neighbourhoods which had the same postcode and combined them into one row and joined the neighbourhoods with a comma inbetween

In [11]:
print(df2)

    Postal Code      Borough  \
0           M1B  Scarborough   
1           M1C  Scarborough   
2           M1E  Scarborough   
3           M1G  Scarborough   
4           M1H  Scarborough   
..          ...          ...   
98          M9N         York   
99          M9P    Etobicoke   
100         M9R    Etobicoke   
101         M9V    Etobicoke   
102         M9W    Etobicoke   

                                         Neighbourhood  
0                                       Rouge, Malvern  
1               Highland Creek, Rouge Hill, Port Union  
2                    Guildwood, Morningside, West Hill  
3                                               Woburn  
4                                            Cedarbrae  
..                                                 ...  
98                                              Weston  
99                                           Westmount  
100  Kingsview Village, Martin Grove Gardens, Richv...  
101  Albion Gardens, Beaumond Heights, Humber

In [12]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

### Identified neighbourhoods which had Not assigned and copied the borough attributes to the Neighbourhood

In [13]:
print(df2)

    Postal Code      Borough  \
0           M1B  Scarborough   
1           M1C  Scarborough   
2           M1E  Scarborough   
3           M1G  Scarborough   
4           M1H  Scarborough   
..          ...          ...   
98          M9N         York   
99          M9P    Etobicoke   
100         M9R    Etobicoke   
101         M9V    Etobicoke   
102         M9W    Etobicoke   

                                         Neighbourhood  
0                                       Rouge, Malvern  
1               Highland Creek, Rouge Hill, Port Union  
2                    Guildwood, Morningside, West Hill  
3                                               Woburn  
4                                            Cedarbrae  
..                                                 ...  
98                                              Weston  
99                                           Westmount  
100  Kingsview Village, Martin Grove Gardens, Richv...  
101  Albion Gardens, Beaumond Heights, Humber

In [14]:
df2.shape

(103, 3)

### Printed the number of rows and columns in the dataframe

In [15]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


### Using geocoder pakage to get Latitude and Longitude values for all Postal Codes in the dataframe

In [16]:
import geocoder # import geocoder

# initialize your variable to None

def get_geocoder(Postal_Code):
# loop until you get the coordinates
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(Postal_Code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [17]:
print (get_geocoder('M1B'))
print (get_geocoder('M1C'))

(43.811525000000074, -79.19551721399995)
(43.78573000000006, -79.15874999999994)


In [18]:
df2['Latitude'], df2['Longitude'] = zip(*df2['Postal Code'].apply(get_geocoder))

In [19]:
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [20]:
df2.shape

(103, 5)

In [21]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


### Using geopy library to get the latitude and longitude values of Toronto

In [22]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Creating a map of Toronto with neighborhoods superimposed on top.

In [24]:
import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [26]:
print(df2)

    Postal Code      Borough  \
0           M1B  Scarborough   
1           M1C  Scarborough   
2           M1E  Scarborough   
3           M1G  Scarborough   
4           M1H  Scarborough   
..          ...          ...   
98          M9N         York   
99          M9P    Etobicoke   
100         M9R    Etobicoke   
101         M9V    Etobicoke   
102         M9W    Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
0                                       Rouge, Malvern  43.811525 -79.195517  
1               Highland Creek, Rouge Hill, Port Union  43.785730 -79.158750  
2                    Guildwood, Morningside, West Hill  43.765690 -79.175256  
3                                               Woburn  43.768359 -79.217590  
4                                            Cedarbrae  43.769688 -79.239440  
..                                                 ...        ...        ...  
98                                              Weston  43.704

### Clustering only the neighborhoods in Scarborough. So Slicing the original dataframe and create a new dataframe of the Scarborough data.

In [27]:
Scarborough_data = df2[df2['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


### Getting the geographical coordinates of Scarborough.

In [29]:
from geopy.geocoders import Nominatim
address = 'Scarborough, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_Scarborough = location.latitude
longitude_Scarborough = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_Scarborough, longitude_Scarborough))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


### Visualizing Scarborough and the neighborhoods in it

In [30]:
import folium
# create map of Scarborough using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude_Scarborough, longitude_Scarborough], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Borough'], Scarborough_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

### Defining Foursquare Credentials and Version

In [31]:
CLIENT_ID = 'THDIHPE40EJKF2DPNEAAVPDKOVGERSCV0OK1TF2SQ0RRGXYT' # your Foursquare ID
CLIENT_SECRET = 'DCVOD3QPAGCKUEUD0Z0ZQFYVEAKY23W2GIBZGLJQSGLJ4IGZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: THDIHPE40EJKF2DPNEAAVPDKOVGERSCV0OK1TF2SQ0RRGXYT
CLIENT_SECRET:DCVOD3QPAGCKUEUD0Z0ZQFYVEAKY23W2GIBZGLJQSGLJ4IGZ


### Exploring the first neighborhood in our dataframe
#### Getting the neighborhood's name

In [33]:
Scarborough_data.loc[0, 'Neighbourhood']

'Rouge, Malvern'

### Getting the neighborhood's latitude and longitude values

In [34]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.811525000000074, -79.19551721399995.


### Getting the top 100 venues that are in Rouge, Malvern within a radius of 500 meters
#### Creating the GET request URL

In [37]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
#-->

#<!--
radius = 500 # define radius
#-->

#<!--
#\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=THDIHPE40EJKF2DPNEAAVPDKOVGERSCV0OK1TF2SQ0RRGXYT&client_secret=DCVOD3QPAGCKUEUD0Z0ZQFYVEAKY23W2GIBZGLJQSGLJ4IGZ&v=20180605&ll=43.811525000000074,-79.19551721399995&radius=500&limit=100'

### GET request and examining the results

In [39]:
import requests
import json
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d87cfed446ea6003906e3e3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.81602500450008,
    'lng': -79.18929289050836},
   'sw': {'lat': 43.80702499550007, 'lng': -79.20174153749154}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5c53f7e18194fc002c76cfbb',
       'name': 'Canadian Appliance Source Whitby',
       'location': {'address': '150 Taunton Rd. W',
        'lat': 43.808353360406464,
        'lng': -79.19133067131041,
        'labeledLatL

### Borrowing the get_category_type function from the Foursquare lab

In [40]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Cleaning the json and structure it into a pandas dataframe

In [42]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Canadian Appliance Source Whitby,Home Service,43.808353,-79.191331


In [43]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


# Explore Neighborhoods in Scarborough

### Function to repeat the same process to all the neighborhoods in Scarborough

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Code to run the above function on each neighborhood and create a new dataframe called Scarborough_venues.

In [46]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighbourhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [47]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(95, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
4,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Heron Park,43.769327,-79.177201,Park


### Number of venues returned for each neighborhood

In [49]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,16,16,16,16,16,16
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",1,1,1,1,1,1
"Birch Cliff, Cliffside West",6,6,6,6,6,6
Cedarbrae,1,1,1,1,1,1
"Clairlea, Golden Mile, Oakridge",11,11,11,11,11,11
"Clarks Corners, Sullivan, Tam O'Shanter",11,11,11,11,11,11
"Cliffcrest, Cliffside, Scarborough Village West",10,10,10,10,10,10
"Dorset Park, Scarborough Town Centre, Wexford Heights",4,4,4,4,4,4
"East Birchmount Park, Ionview, Kennedy Park",5,5,5,5,5,5


### Unique categories from all the returned venues

In [50]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 57 uniques categories.


# Analyzing Each Neighborhood

In [51]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bar,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,...,Shanghai Restaurant,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
Scarborough_onehot.shape

(95, 58)

### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [53]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_grouped

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bar,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,...,Shanghai Restaurant,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.0625,0.062500,0.0,0.000000,0.0625,0.000000,0.000000,0.000000,...,0.0625,0.125000,0.000000,0.000000,0.125,0.0625,0.000000,0.000000,0.00,0.0625
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.0000
2,"Birch Cliff, Cliffside West",0.0,0.0000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.166667,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.0000
3,Cedarbrae,0.0,0.0000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.0000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.0000,0.181818,0.0,0.000000,0.0000,0.000000,0.181818,0.090909,...,0.0000,0.000000,0.000000,0.090909,0.000,0.0000,0.000000,0.000000,0.00,0.0000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.0000,0.000000,0.0,0.090909,0.0000,0.000000,0.000000,0.000000,...,0.0000,0.090909,0.000000,0.000000,0.000,0.0000,0.090909,0.000000,0.00,0.0000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.0,0.0000,0.000000,0.0,0.000000,0.0000,0.100000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.0000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.0000,0.250000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.0000
8,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.0000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.0000
9,"Guildwood, Morningside, West Hill",0.0,0.0000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.00,0.0000


In [54]:
Scarborough_grouped.shape

(16, 58)

### Printing each neighborhood along with the top 5 most common venues

In [55]:
num_top_venues = 5

for hood in Scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0     Chinese Restaurant  0.12
1          Shopping Mall  0.12
2            Supermarket  0.12
3  Vietnamese Restaurant  0.06
4        Badminton Court  0.06


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0           Pharmacy   1.0
1        Auto Garage   0.0
2       Home Service   0.0
3  Indian Restaurant   0.0
4       Intersection   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0               Gym Pool  0.17
1                    Gym  0.17
2           Skating Rink  0.17
3  General Entertainment  0.17
4                   Park  0.17


----Cedarbrae----
               venue  freq
0         Playground   1.0
1        Auto Garage   0.0
2       Home Service   0.0
3  Indian Restaurant   0.0
4       Intersection   0.0


----Clairlea, Golden Mile, Oakridge----
           venue  freq
0   Intersection  0.18
1         Bakery  0.18
2    Coffee Shop  0.18
3       Bus Line  0.18
4  M

### Function to sort the venues in descending order.

In [85]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Creating the new dataframe and display the top 10 venues for each neighborhood.

In [116]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
       neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Supermarket,Shopping Mall,Vietnamese Restaurant,Shanghai Restaurant,Hong Kong Restaurant,Park,Pool,Bubble Tea Shop,Grocery Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff, Cliffside West",Gym,General Entertainment,Skating Rink,Park,Gym Pool,College Stadium,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Gift Shop
3,Cedarbrae,Playground,College Stadium,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,"Clairlea, Golden Mile, Oakridge",Intersection,Bakery,Coffee Shop,Bus Line,Bus Station,Metro Station,Soccer Field,Electronics Store,Fast Food Restaurant,Fried Chicken Joint


In [91]:
np.arange(Scarborough_grouped.shape[0])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

# Clustering Neighborhoods

### Running k-means to cluster the neighborhood into 5 clusters

In [117]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [118]:

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Supermarket,Shopping Mall,Vietnamese Restaurant,Shanghai Restaurant,Hong Kong Restaurant,Park,Pool,Bubble Tea Shop,Grocery Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff, Cliffside West",Gym,General Entertainment,Skating Rink,Park,Gym Pool,College Stadium,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Gift Shop
3,Cedarbrae,Playground,College Stadium,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,"Clairlea, Golden Mile, Oakridge",Intersection,Bakery,Coffee Shop,Bus Line,Bus Station,Metro Station,Soccer Field,Electronics Store,Fast Food Restaurant,Fried Chicken Joint


### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [119]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged = Scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,2.0,Home Service,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,3.0,Bar,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256,0.0,Construction & Landscaping,Gym / Fitness Center,Park,Vietnamese Restaurant,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.768359,-79.217590,0.0,Coffee Shop,Park,Soccer Field,Korean Restaurant,Business Service,Vietnamese Restaurant,Gym,Grocery Store,Gift Shop,General Entertainment
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,1.0,Playground,College Stadium,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [122]:
Scarborough_merged.shape

(17, 16)

In [129]:
Scarborough_merged.dropna(inplace=True)
Scarborough_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,2.0,Home Service,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,3.0,Bar,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256,0.0,Construction & Landscaping,Gym / Fitness Center,Park,Vietnamese Restaurant,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.768359,-79.217590,0.0,Coffee Shop,Park,Soccer Field,Korean Restaurant,Business Service,Vietnamese Restaurant,Gym,Grocery Store,Gift Shop,General Entertainment
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,1.0,Playground,College Stadium,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750,0.0,Train Station,Grocery Store,Indian Restaurant,Restaurant,Vietnamese Restaurant,Construction & Landscaping,Gym,Gift Shop,General Entertainment,Fried Chicken Joint
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670,0.0,Discount Store,Department Store,Convenience Store,Coffee Shop,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055,0.0,Intersection,Bakery,Coffee Shop,Bus Line,Bus Station,Metro Station,Soccer Field,Electronics Store,Fast Food Restaurant,Fried Chicken Joint
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976,0.0,Fast Food Restaurant,Burger Joint,Coffee Shop,Liquor Store,Discount Store,Sandwich Place,Pizza Place,Hardware Store,Pharmacy,Fried Chicken Joint
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163,0.0,Gym,General Entertainment,Skating Rink,Park,Gym Pool,College Stadium,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Gift Shop


In [130]:
Scarborough_merged.shape

(16, 16)

In [131]:
cols = ['Cluster Labels']
Scarborough_merged[cols] = Scarborough_merged[cols].applymap(np.int64)

In [132]:
Scarborough_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,2,Home Service,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,3,Bar,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256,0,Construction & Landscaping,Gym / Fitness Center,Park,Vietnamese Restaurant,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.768359,-79.217590,0,Coffee Shop,Park,Soccer Field,Korean Restaurant,Business Service,Vietnamese Restaurant,Gym,Grocery Store,Gift Shop,General Entertainment
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,1,Playground,College Stadium,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750,0,Train Station,Grocery Store,Indian Restaurant,Restaurant,Vietnamese Restaurant,Construction & Landscaping,Gym,Gift Shop,General Entertainment,Fried Chicken Joint
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670,0,Discount Store,Department Store,Convenience Store,Coffee Shop,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055,0,Intersection,Bakery,Coffee Shop,Bus Line,Bus Station,Metro Station,Soccer Field,Electronics Store,Fast Food Restaurant,Fried Chicken Joint
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976,0,Fast Food Restaurant,Burger Joint,Coffee Shop,Liquor Store,Discount Store,Sandwich Place,Pizza Place,Hardware Store,Pharmacy,Fried Chicken Joint
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163,0,Gym,General Entertainment,Skating Rink,Park,Gym Pool,College Stadium,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Gift Shop


### Visualizing the resulting clusters

In [133]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
from matplotlib import cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighbourhood'], Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [134]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0,Construction & Landscaping,Gym / Fitness Center,Park,Vietnamese Restaurant,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
3,Scarborough,0,Coffee Shop,Park,Soccer Field,Korean Restaurant,Business Service,Vietnamese Restaurant,Gym,Grocery Store,Gift Shop,General Entertainment
5,Scarborough,0,Train Station,Grocery Store,Indian Restaurant,Restaurant,Vietnamese Restaurant,Construction & Landscaping,Gym,Gift Shop,General Entertainment,Fried Chicken Joint
6,Scarborough,0,Discount Store,Department Store,Convenience Store,Coffee Shop,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop
7,Scarborough,0,Intersection,Bakery,Coffee Shop,Bus Line,Bus Station,Metro Station,Soccer Field,Electronics Store,Fast Food Restaurant,Fried Chicken Joint
8,Scarborough,0,Fast Food Restaurant,Burger Joint,Coffee Shop,Liquor Store,Discount Store,Sandwich Place,Pizza Place,Hardware Store,Pharmacy,Fried Chicken Joint
9,Scarborough,0,Gym,General Entertainment,Skating Rink,Park,Gym Pool,College Stadium,Vietnamese Restaurant,Construction & Landscaping,Grocery Store,Gift Shop
10,Scarborough,0,Bakery,Gift Shop,Light Rail Station,Rental Service,Vietnamese Restaurant,Construction & Landscaping,Gym,Grocery Store,General Entertainment,Fried Chicken Joint
12,Scarborough,0,Chinese Restaurant,Supermarket,Shopping Mall,Vietnamese Restaurant,Shanghai Restaurant,Hong Kong Restaurant,Park,Pool,Bubble Tea Shop,Grocery Store
13,Scarborough,0,Pizza Place,Shopping Mall,Fried Chicken Joint,Coffee Shop,Pharmacy,Chinese Restaurant,Bus Stop,Hobby Shop,Breakfast Spot,Thai Restaurant


### Cluster 2

In [135]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,1,Playground,College Stadium,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### CLuster 3

In [136]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Home Service,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


### Cluster 4

In [137]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 3, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Bar,Vietnamese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


### Cluster 5

In [138]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 4, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,4,Auto Garage,Hardware Store,Gym / Fitness Center,Gym,Grocery Store,Gift Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
